In [1]:
import bs4
import requests
import re
import random
import os
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import scipy as sc
import numpy as np
from numpy import random
from time import sleep
import wget
from collections import defaultdict
from random import randint, choice


In [243]:
def scrape_matches():
    teamscore_1=[]
    teamname_1=[]
    teamname_2=[]
    teamscore_2=[]
  
    path="D:\htlv"
    
    d=[]
    file_list = [os.path.join(path, file_name) for file_name in next(os.walk(path))[2]]
    for file in file_list:
        with open(file, encoding="utf8") as match_file:
            soup=BeautifulSoup(match_file, 'html.parser')
            
            team_1=soup.find('div', attrs={'class':'team-left'})
            team_2= soup.find('div',attrs={'class':'team-right'})
            
            teamname_1=team_1.find('a', {'class': 'block text-ellipsis'})
            teamname_2=team_2.find('a', {'class': 'block text-ellipsis'})
            
            match_date=soup.find('span', {'data-time-format':'yyyy-MM-dd HH:mm'}).string
            
            match_map = soup.find(text='Map').next.strip()
          
            
            teamscore_1 = int(team_1.find('div', {'class': re.compile('bold.*')}).string)
            teamscore_2 = int(team_2.find('div', {'class': re.compile('bold.*')}).string)
            
            match_result = {'date':match_date, 'team1': teamname_1.string, 'team1_score': teamscore_1, 'team2': teamname_2.string, 'team2_score': teamscore_2, 'winner': (teamname_1.string if max(teamscore_1, teamscore_2)==teamscore_1 else teamname_2.string), 'map': match_map}
            d.append(match_result)
            
    match_df=pd.DataFrame(d)
            
            
    match_file.close()
    return match_df

In [345]:
match_df=scrape_matches()
match_df.to_csv('Matches.csv')
match_df

,date,team1,team1_score,team2,team2_score,winner,map
0,2021-12-16 10:00,1620 Kings,16,8Ballers,7,1620 Kings,Dust2
1,2021-12-15 10:00,Paradox,16,1620 Kings,7,Paradox,Nuke
2,2022-01-19 16:30,1shot,7,Astralis Talent,16,Astralis Talent,Vertigo
3,2022-01-22 13:10,1shot,16,D13,9,1shot,Nuke
4,2022-01-16 19:15,DBL PONEY,16,1shot,8,DBL PONEY,Vertigo
...,...,...,...,...,...,...,...
994,2022-01-23 14:00,Young Ninjas,7,Copenhagen Flames,16,Copenhagen Flames,Dust2
995,2022-01-05 17:00,fnatic Rising,11,Young Ninjas,16,Young Ninjas,Overpass
996,2022-01-22 10:00,LPSP,16,Young Ninjas,10,LPSP,Nuke
997,2022-01-11 21:10,Young Ninjas,19,NAVI Junior,15,Young Ninjas,Inferno


In [253]:
def scrape_match_stats():
    path="D:\htlv"
    
  
    best_rating=" "
    most_kills=" "
    most_dmg=" "
    most_A=" "
    most_awp_kills=" "
    most_Fkills=" "
    
    
    stats_df=[]
    file_list = [os.path.join(path, file_name) for file_name in next(os.walk(path))[2]]
    for file in file_list:
        with open(file, encoding="utf8") as match_file:
            soup=BeautifulSoup(match_file, 'html.parser')
            
           
            
            Team=soup.find('th',{'class':"st-teamname text-ellipsis"}).text
            #print(Team)
            Player=soup.find('td',{'class':"st-player"}).text.strip()
            #print(Player)
          
            kills=soup.find('td',{'class':"st-kills"}).text.split(' ')[0]
            
            assists=soup.find('td',{'class':"st-assists"}).text.split(' ')[0]
            
            deaths=soup.find('td',{'class':"st-deaths"}).text.split(' ')[0]
            
            kill_assist_survive_trade=soup.find('td',{'class':"st-kdratio"}).text
           
            adr=soup.find('td',{'class':"st-adr"}).text
           
            player_rating=soup.find('td',{'class':"st-rating"}).text
           
            most=soup.findAll('span',{'class':'valueName'})
    
            
            most_names = soup.findAll('div', {'class': 'name'})
            most_names_list=[name.text for name in most_names]
            
           
            most_names_anchor =[anchor.text for anchor in soup.findAll('a', {'href': re.compile('/stats/players/*')}) if anchor.text != 'Go to Player'][:6]
            
            for i, (n,m) in enumerate(zip(most_names_anchor,most)):
                
                if i==0:
                    if Player ==n:
                        most_kills=m.text
                    else:
                        most_kills=np.NaN
                elif i==1:
                    if Player ==n:
                        most_dmg=m.text
                    else:
                        most_dmg=np.NaN
                elif i==2:
                    if Player==n:
                        most_A=m.text
                    else:
                        most_A=np.NaN
                elif i==3:
                    if Player==n:
                        most_awp_kills=m.text
                    else:
                        most_awp_kills=np.NaN
                elif i == 4:
                    if Player == n:
                        most_Fkills=m.text
                    else:
                        most_Fkills=np.NaN
                elif i == 5:
                    if Player == n:
                        best_rating=m.text
                    else:
                        best_rating=np.NaN
    
            
            df_dict={"team":Team, "Player":Player,"K":kills,"A":assists,"D":deaths,"KAST":kill_assist_survive_trade,"ADR":adr,"Rating":player_rating,"Best Rating":best_rating,"Most Kills":most_kills,"Most Damage":most_dmg,"Most Assists":most_A,"Most AWP Kills":most_awp_kills,"Most First Kills":most_Fkills}
             
            
            stats_df.append(df_dict)
            
    df=pd.DataFrame(stats_df)
        
    match_file.close()
    return df

In [346]:
df=scrape_match_stats()
df.to_csv('PlayerInfoPerMatch.csv')
df

,team,Player,K,A,D,KAST,ADR,Rating,Best Rating,Most Kills,Most Damage,Most Assists,Most AWP Kills,Most First Kills
0,1620 Kings,cookie,25,7,9,82.6%,123.7,1.90,1.90,25,123.7,7,14,NaN
1,Paradox,chelleos,21,5,16,78.3%,89.9,1.43,1.43,21,NaN,NaN,8,6
2,1shot,Synyx,18,0,20,52.2%,85.0,0.89,NaN,NaN,NaN,NaN,NaN,NaN
3,1shot,faydett,24,5,13,88.0%,102.5,1.51,1.51,NaN,NaN,NaN,NaN,NaN
4,DBL PONEY,afro,26,3,13,75.0%,96.1,1.55,1.55,26,NaN,NaN,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,Young Ninjas,Sapec,19,2,17,65.2%,79.5,1.08,NaN,NaN,NaN,NaN,NaN,NaN
995,fnatic Rising,Peppzor,20,2,21,70.4%,75.8,1.08,NaN,NaN,NaN,NaN,NaN,NaN
996,LPSP,NaToSaphiX,24,4,12,84.6%,93.4,1.60,1.60,24,93.4,NaN,8,NaN
997,Young Ninjas,Ro1f,32,0,21,85.3%,91.2,1.39,1.39,32,NaN,NaN,NaN,NaN


In [313]:
df.shape

(999, 14)

In [314]:
df.isnull().sum()

team                  0
Player                0
K                     0
A                     0
D                     0
KAST                  0
ADR                   0
Rating                0
Best Rating         478
Most Kills          559
Most Damage         581
Most Assists        885
Most AWP Kills      826
Most First Kills    720
dtype: int64

In [315]:
df.isnull().values.any()

True

In [340]:
def remove_corrupt_rows(df):
    new_df=df.dropna(axis=0,how='any',thresh=9,inplace=False)
    return new_df

In [341]:
df_corrupted_removed = remove_corrupt_rows(df)
print(df_corrupted_removed)

                team      Player   K  A   D   KAST    ADR Rating Best Rating  \
0         1620 Kings      cookie  25  7   9  82.6%  123.7   1.90        1.90   
1            Paradox    chelleos  21  5  16  78.3%   89.9   1.43        1.43   
3              1shot     faydett  24  5  13  88.0%  102.5   1.51        1.51   
4          DBL PONEY        afro  26  3  13  75.0%   96.1   1.55        1.55   
5    Into the Breach      CYPHER  33  3  20  75.0%  123.6   1.79        1.79   
..               ...         ...  .. ..  ..    ...    ...    ...         ...   
992     Young Ninjas        phzy  21  7  22  75.0%   75.6   1.02         NaN   
993     Young Ninjas       Sapec  21  9  22  80.0%   99.9   1.21         NaN   
996             LPSP  NaToSaphiX  24  4  12  84.6%   93.4   1.60        1.60   
997     Young Ninjas        Ro1f  32  0  21  85.3%   91.2   1.39        1.39   
998     Young Ninjas    Jackinho  18  5  21  66.7%   80.3   1.11         NaN   

    Most Kills Most Damage Most Assists

In [342]:
def replace_NaN_values(df_corrupted_removed):
    df_corrupted_removed=df_corrupted_removed.fillna(0)
    return df_corrupted_removed

In [344]:
df_replaced_nan= replace_NaN_values(df)
print(df_replaced_nan)

              team      Player   K  A   D   KAST    ADR Rating Best Rating  \
0       1620 Kings      cookie  25  7   9  82.6%  123.7   1.90        1.90   
1          Paradox    chelleos  21  5  16  78.3%   89.9   1.43        1.43   
2            1shot       Synyx  18  0  20  52.2%   85.0   0.89           0   
3            1shot     faydett  24  5  13  88.0%  102.5   1.51        1.51   
4        DBL PONEY        afro  26  3  13  75.0%   96.1   1.55        1.55   
..             ...         ...  .. ..  ..    ...    ...    ...         ...   
994   Young Ninjas       Sapec  19  2  17  65.2%   79.5   1.08           0   
995  fnatic Rising     Peppzor  20  2  21  70.4%   75.8   1.08           0   
996           LPSP  NaToSaphiX  24  4  12  84.6%   93.4   1.60        1.60   
997   Young Ninjas        Ro1f  32  0  21  85.3%   91.2   1.39        1.39   
998   Young Ninjas    Jackinho  18  5  21  66.7%   80.3   1.11           0   

    Most Kills Most Damage Most Assists Most AWP Kills Most Fir